In [42]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func

In [43]:
spark = SparkSession.builder.appName("sparksql").getOrCreate()

In [44]:
print(spark.version)

3.5.4


In [45]:
#This format is used when there's a header and use to format the columns
df_operations_management = spark.read.format("csv").\
                                option("inferSchema", "true").\
                                option("header", "true").\
                                option("path", "/app/operations_management.csv").\
                                load()

df_operations_management.printSchema()

root
 |-- description: string (nullable = true)
 |-- industry: string (nullable = true)
 |-- level: integer (nullable = true)
 |-- size: string (nullable = true)
 |-- line_code: string (nullable = true)
 |-- value: integer (nullable = true)



In [46]:
df_2 = df_operations_management.select("industry", "value").\
                               filter(func.col("value") > 10000).\
                               orderBy(func.desc("value"))

df_2.printSchema()

df_2.show(5)

root
 |-- industry: string (nullable = true)
 |-- value: integer (nullable = true)

+--------+-----+
|industry|value|
+--------+-----+
|   total|41091|
|   total|40431|
|   total|33984|
|   total|33750|
|   total|32652|
+--------+-----+
only showing top 5 rows



In [49]:
df_2.createOrReplaceTempView("data")
spark.sql('''
        SELECT *
        FROM data
        WHERE value > 10000
        LIMIT 5
''').show()

+--------+-----+
|industry|value|
+--------+-----+
|   total|41091|
|   total|40431|
|   total|33984|
|   total|33750|
|   total|32652|
+--------+-----+



In [ ]:
df_3 = df_operations_management.select("industry", "value").\
                               filter((func.col("industry") != "total") & (func.col("value") > 200)).\
                               orderBy(func.desc("value"))

df_3.printSchema()

df_3.show(5)

root
 |-- industry: string (nullable = true)
 |-- value: integer (nullable = true)

+--------------------+-----+
|            industry|value|
+--------------------+-----+
|        Construction| 6030|
|        Construction| 5904|
|        Construction| 5229|
|Accommodation & f...| 5058|
|        Construction| 4965|
+--------------------+-----+
only showing top 5 rows



In [51]:
df_3.createOrReplaceTempView("data3")
spark.sql('''
        SELECT *
        FROM data3
        WHERE value > 200 AND NOT industry = "total"
        LIMIT 5
''').show()

+--------------------+-----+
|            industry|value|
+--------------------+-----+
|        Construction| 6030|
|        Construction| 5904|
|        Construction| 5229|
|Accommodation & f...| 5058|
|        Construction| 4965|
+--------------------+-----+

